In [1]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 6.5MB/s 


In [1]:
import json
import requests
import numpy as np
import pandas as pd

In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [4]:
sentences = ["VADER is smart, handsome, and funny.",  # positive sentence example
             "VADER is smart, handsome, and funny!",  # punctuation emphasis handled correctly (sentiment intensity adjusted)
             "VADER is very smart, handsome, and funny.", # booster words handled correctly (sentiment intensity adjusted)
             "VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
             "VADER is VERY SMART, handsome, and FUNNY!!!", # combination of signals - VADER appropriately adjusts intensity
             "VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!", # booster words & punctuation make this close to ceiling for score
             "VADER is not smart, handsome, nor funny.",  # negation sentence example
             "The book was good.",  # positive sentence
             "At least it isn't a horrible book.",  # negated negative sentence with contraction
             "The book was only kind of good.", # qualified positive sentence is handled correctly (intensity adjusted)
             "The plot was good, but the characters are uncompelling and the dialog is not great.", # mixed negation sentence
             "Today SUX!",  # negative slang with capitalization emphasis
             "Today only kinda sux! But I'll get by, lol", # mixed sentiment example with slang and constrastive conjunction "but"
             "Make sure you :) or :D today!",  # emoticons handled
             "Catch utf-8 emoji such as such as 💘 and 💋 and 😁",  # emojis handled
             "Not bad at all"  # Capitalized negation
             ]

analyzer = SentimentIntensityAnalyzer()
for sentence in sentences:
    vs = analyzer.polarity_scores(sentence)
    print("{:-<65} {}".format(sentence, str(vs)))

VADER is smart, handsome, and funny.----------------------------- {'neg': 0.0, 'neu': 0.254, 'pos': 0.746, 'compound': 0.8316}
VADER is smart, handsome, and funny!----------------------------- {'neg': 0.0, 'neu': 0.248, 'pos': 0.752, 'compound': 0.8439}
VADER is very smart, handsome, and funny.------------------------ {'neg': 0.0, 'neu': 0.299, 'pos': 0.701, 'compound': 0.8545}
VADER is VERY SMART, handsome, and FUNNY.------------------------ {'neg': 0.0, 'neu': 0.246, 'pos': 0.754, 'compound': 0.9227}
VADER is VERY SMART, handsome, and FUNNY!!!---------------------- {'neg': 0.0, 'neu': 0.233, 'pos': 0.767, 'compound': 0.9342}
VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!--------- {'neg': 0.0, 'neu': 0.294, 'pos': 0.706, 'compound': 0.9469}
VADER is not smart, handsome, nor funny.------------------------- {'neg': 0.646, 'neu': 0.354, 'pos': 0.0, 'compound': -0.7424}
The book was good.----------------------------------------------- {'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'co

In [5]:
vs

{'compound': 0.431, 'neg': 0.0, 'neu': 0.513, 'pos': 0.487}

In [6]:
!ls

drive  sample_data


In [7]:
neg_tweets = []
with open('drive/MyDrive/BuildUp/negative_tweets_arabic_20181207_100.txt', encoding='utf-8') as fh:
    neg_tweets = fh.read()

In [8]:
type(neg_tweets)

str

In [9]:
neg_tweets[:100]

'لـآ شـيء يشـبه شيء . . ولـآ أحد يعو ّ ض غيآب أحد 💔 أشياؤنـآ ﺂلم ُ ختلفـة إن ضـآعت . . لـيس َ لهـآ بـ'

In [11]:
neg_tweets[:1000]

['لـآ شـيء يشـبه شيء . . ولـآ أحد يعو ّ ض غيآب أحد 💔 أشياؤنـآ ﺂلم ُ ختلفـة إن ضـآعت . . لـيس َ لهـآ بـديـل 💭 🔕 🔹 🔹\n',
 '🕊 📨 ⇣ " بــ ؏ ــض ُ آلـبـش َ ــر أخــلآقـه ُ ــم فـي غـيـب ُ ــوبـهہ مــدى آلح َ ـيــاهہ • " 📨 🔕\n',
 'سوي اي حاجه بس لاتتكلم على شخص بسوء من وراه يكفي انه وثق فيك وباح لك بأسراره ممكن يكون غبي لأنه وثق فيك 🔕 👌 …\n',
 '، فارقتني ؟ طيب وإذا فارقتني ؟ لا حس ّ بـ موادع ! ولا اشعر بـ انكسار ! ! واذا على دمعي ، أبد ما اوجعتني بعض الدموع تطيح بـ …\n',
 '⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ٓ ٰ تبا لبعض البشر اشعروني بأن طيبتي هي أسوء ما أملك 🔕 💔 💭 ٓ ⠀ ⠀ ⠀\n',
 '؛ ؛ ؛ ؛ ؛ ؛ ؛ ؛ ؛ ؛ . ويبقى موعدك ( سي ِ ّ د ) مواعيدي , وتبقى ( أنت ) متسي ّ د على الساده 🔕 💭 🎻\n',
 'ان بغيت المواصل أو نويت الوداع باب قلبي مشر ّ ع وإنت راعي محل 🔕\n',
 'ان بغيت المواصل أو نويت الوداع باب قلبي مشر ّ ع وإنت راعي محل 🔕\n',
 'الشيء الوحيد الذي تميزت به دوما ً ، أنني ﻻ أخون رغم خيانة الناس لي ، ﻻ يروق لي رد السيئة بالسيئة بل أكتفي بالرحيل واﻹبتعاد عن من خ …\n',
 '┋ ❁ ˛ ˛ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ عز ّ يت نفسي عن بشر 

In [12]:
neg_tweets[0]

'لـآ شـيء يشـبه شيء . . ولـآ أحد يعو ّ ض غيآب أحد 💔 أشياؤنـآ ﺂلم ُ ختلفـة إن ضـآعت . . لـيس َ لهـآ بـديـل 💭 🔕 🔹 🔹\n'

In [10]:
neg_tweets = []
with open('drive/MyDrive/BuildUp/negative_tweets_arabic_20181207_100.txt', encoding='utf-8') as fh:
    tweet = fh.readline()

    cnt = 1
    while tweet:
        # print("Line {}: {}".format(cnt, tweet.strip()))
        neg_tweets.append(tweet)
        tweet = fh.readline()
        cnt += 1

print(cnt, len(neg_tweets))

7393 7392


In [ ]:
print(neg_tweets[:1000])

لـآ شـيء يشـبه شيء . . ولـآ أحد يعو ّ ض غيآب أحد 💔 أشياؤنـآ ﺂلم ُ ختلفـة إن ضـآعت . . لـيس َ لهـآ بـديـل 💭 🔕 🔹 🔹
🕊 📨 ⇣ " بــ ؏ ــض ُ آلـبـش َ ــر أخــلآقـه ُ ــم فـي غـيـب ُ ــوبـهہ مــدى آلح َ ـيــاهہ • " 📨 🔕
سوي اي حاجه بس لاتتكلم على شخص بسوء من وراه يكفي انه وثق فيك وباح لك بأسراره ممكن يكون غبي لأنه وثق فيك 🔕 👌 …
، فارقتني ؟ طيب وإذا فارقتني ؟ لا حس ّ بـ موادع ! ولا اشعر بـ انكسار ! ! واذا على دمعي ، أبد ما اوجعتني بعض الدموع تطيح بـ …
⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ ٓ ٰ تبا لبعض البشر اشعروني بأن طيبتي هي أسوء ما أملك 🔕 💔 💭 ٓ ⠀ ⠀ ⠀
؛ ؛ ؛ ؛ ؛ ؛ ؛ ؛ ؛ ؛ . ويبقى موعدك ( سي ِ ّ د ) مواعيدي , وتبقى ( أنت ) متسي ّ د على الساده 🔕 💭 🎻
ان بغيت المواصل أو نويت الوداع باب قلبي مشر ّ ع وإنت راعي محل 🔕
ان بغيت المواصل أو نويت الوداع باب قلبي مشر ّ ع وإنت راعي محل 🔕
الشيء الوحيد الذي تميزت به دوما ً ، أنني ﻻ أخون رغم خيانة الناس لي ، ﻻ يروق لي رد السيئة بالسيئة بل أكتفي بالرحيل واﻹبتعاد عن من خ …
┋ ❁ ˛ ˛ ⠀ ⠀ ⠀ ⠀ ⠀ ⠀ عز ّ يت نفسي عن بشر ظنهم شــين ! مهمــا رفعت ، ق َ ل أدبهم . . ⠀ ⠀ ⠀ ⠀ ⠀ ⁽ 🔕

In [ ]:
!ls

drive
negative_tweets_arabic_20181207_100.txt
positive_tweets_arabic_20181207_100.txt
sample_data


In [15]:
df = pd.read_json('drive/MyDrive/BuildUp/negative_tweets_arabic_20181207_100.json', lines=True)

In [16]:
df.head()

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,withheld_in_countries
0,2018-12-07 11:37:03+00:00,1071005663369814016,1071005663369814016,RT @ggeewww123: لـآ شـيء يشـبه شيء.. \nولـآ أح...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'ar', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 858138066019655680, 'id_str': '85813806...",NaN,NaN,None,NaN,{'created_at': 'Fri Dec 07 08:27:14 +0000 2018...,False,7,0,False,False,ar,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-12-07 11:36:27+00:00,1071005515172450306,1071005515172450304,"RT @moneefNa: 🕊📨⇣\n\n""بــ؏ــضُ آلـبـشَــر أخــ...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'ar', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 1545924488, 'id_str': '1545924488', 'na...",NaN,NaN,None,NaN,{'created_at': 'Wed Dec 05 20:43:01 +0000 2018...,False,920,0,False,False,ar,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-12-07 11:35:12+00:00,1071005201409163264,1071005201409163264,RT @theshdwa1001: #theshdwa1001 \nسوي اي حاجه ...,False,"{'hashtags': [{'text': 'theshdwa1001', 'indice...","{'iso_language_code': 'ar', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 797185018787340288, 'id_str': '79718501...",NaN,NaN,None,NaN,{'created_at': 'Fri Dec 07 07:19:13 +0000 2018...,False,19,0,False,False,ar,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-12-07 11:35:03+00:00,1071005162469253120,1071005162469253120,RT @toott4321: ،\n\nفارقتني ؟\nطيب وإذا فارقتن...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'ar', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 868550716197089282, 'id_str': '86855071...",NaN,NaN,None,NaN,{'created_at': 'Thu Dec 06 18:58:55 +0000 2018...,False,55,0,False,False,ar,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-12-07 11:34:19+00:00,1071004979081740288,1071004979081740288,RT @jood_7mA: ⠀\n\nㅤ⠀⠀⠀⠀⠀⠀⠀ ⠀⠀\n\n ⠀⠀⠀⠀⠀⠀⠀ \nٓ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'ar', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 966365121265233920, 'id_str': '96636512...",NaN,NaN,None,NaN,{'created_at': 'Sat Dec 01 15:48:34 +0000 2018...,False,88,0,False,False,ar,"{'media': [{'id': 1068894626952749061, 'id_str...",0.0,NaN,NaN,NaN,NaN


In [17]:
df.columns

Index(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities',
       'metadata', 'source', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo',
       'coordinates', 'place', 'contributors', 'retweeted_status',
       'is_quote_status', 'retweet_count', 'favorite_count', 'favorited',
       'retweeted', 'lang', 'extended_entities', 'possibly_sensitive',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status',
       'withheld_in_countries'],
      dtype='object')

In [18]:
df.shape

(18041, 31)

In [19]:
df.loc[0, 'text']

'RT @ggeewww123: لـآ شـيء يشـبه شيء.. \nولـآ أحد يعوّض غيآب أحد 💔\nأشياؤنـآ ﺂلمُختلفـة إن ضـآعت..\n لـيسَ لهـآ بـديـل💭🔕 \n\n🔹 🔹'

In [3]:
filenames = ['drive/MyDrive/BuildUp/negative_tweets_arabic_20181207_100.json',
             'drive/MyDrive/BuildUp/positive_tweets_arabic_20181207_100.json']

tweets = []
data = []
for filename in filenames:
    data.append(pd.read_json(filename, lines=True))
    for line in open(filename, 'r', encoding='utf-8'):
        tweets.append(json.loads(line))

df = pd.json_normalize(tweets)

In [21]:
df.shape

(38398, 348)

In [22]:
data[0].shape, data[1].shape

((18041, 31), (20357, 30))

In [23]:
data[0].columns

Index(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities',
       'metadata', 'source', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo',
       'coordinates', 'place', 'contributors', 'retweeted_status',
       'is_quote_status', 'retweet_count', 'favorite_count', 'favorited',
       'retweeted', 'lang', 'extended_entities', 'possibly_sensitive',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status',
       'withheld_in_countries'],
      dtype='object')

In [24]:
data[1].columns

Index(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities',
       'metadata', 'source', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo',
       'coordinates', 'place', 'contributors', 'retweeted_status',
       'is_quote_status', 'retweet_count', 'favorite_count', 'favorited',
       'retweeted', 'lang', 'extended_entities', 'possibly_sensitive',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status'],
      dtype='object')

In [26]:
set(data[0].columns) - (set(data[1].columns))

{'withheld_in_countries'}

In [28]:
for col in df.columns:
    if 'withheld_in_countries' in col:
        print(col)

withheld_in_countries
user.withheld_in_countries


In [29]:
s = 'withheld_in_countries'
for i in range(len(data[1])):
    for col in data[1].columns:
        try:
            if s in set(data[1].loc[i, col].keys()):
                print(i)
                break
        except:
            pass

In [30]:
s = 'withheld_in_countries'
for i in range(len(data[0])):
    for col in data[0].columns:
        try:
            if s in set(data[0].loc[i, col].keys()):
                print(i)
                break
        except:
            pass

2220
8984


In [31]:
data[0].iloc[2220]

created_at                                           2018-12-07 11:10:52+00:00
id                                                         1070999074629976064
id_str                                                     1070999074629976064
text                         RT @aljaber777: @_AgRa3kw_ لعمرك ما تُبلى سراب...
truncated                                                                False
entities                     {'hashtags': [], 'symbols': [], 'user_mentions...
metadata                     {'iso_language_code': 'ar', 'result_type': 're...
source                       <a href="http://twitter.com/download/iphone" r...
in_reply_to_status_id                                                      NaN
in_reply_to_status_id_str                                                  NaN
in_reply_to_user_id                                                        NaN
in_reply_to_user_id_str                                                    NaN
in_reply_to_screen_name                             

In [32]:
data[0].iloc[2220].user

{'contributors_enabled': False,
 'created_at': 'Mon May 12 23:57:05 +0000 2014',
 'default_profile': True,
 'default_profile_image': False,
 'description': 'وَكُلُّهُمْ آَتِيهِ يَوْمَ الْقِيَامَةِ فَرْدًا',
 'entities': {'description': {'urls': []}},
 'favourites_count': 1017,
 'follow_request_sent': False,
 'followers_count': 41115,
 'following': False,
 'friends_count': 193,
 'geo_enabled': False,
 'has_extended_profile': False,
 'id': 2492125436,
 'id_str': '2492125436',
 'is_translation_enabled': False,
 'is_translator': False,
 'lang': 'en',
 'listed_count': 80,
 'location': '101',
 'name': 'الاقـــرع',
 'notifications': False,
 'profile_background_color': 'C0DEED',
 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_background_tile': False,
 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/2492125436/1540442204',
 'profile_image_url': 

In [33]:
data[0].iloc[0].user

{'contributors_enabled': False,
 'created_at': 'Sat Apr 29 01:57:13 +0000 2017',
 'default_profile': True,
 'default_profile_image': False,
 'description': '#الهلال #مدريد',
 'entities': {'description': {'urls': []}},
 'favourites_count': 52,
 'follow_request_sent': False,
 'followers_count': 4499,
 'following': False,
 'friends_count': 3024,
 'geo_enabled': False,
 'has_extended_profile': False,
 'id': 858138066019655680,
 'id_str': '858138066019655680',
 'is_translation_enabled': False,
 'is_translator': False,
 'lang': 'ar',
 'listed_count': 2,
 'location': '',
 'name': 'محظوووره رتويت',
 'notifications': False,
 'profile_background_color': 'F5F8FA',
 'profile_background_image_url': None,
 'profile_background_image_url_https': None,
 'profile_background_tile': False,
 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/858138066019655680/1542669750',
 'profile_image_url': 'http://pbs.twimg.com/profile_images/1065216038449700864/oLODSear_normal.jpg',
 'profile_image_url_http

In [34]:
data[1].iloc[0].user

{'contributors_enabled': False,
 'created_at': 'Fri Jan 18 22:35:10 +0000 2013',
 'default_profile': True,
 'default_profile_image': False,
 'description': '\u200f\u200f\u200f\u200f\u200fقل اعوذ برب الفلق',
 'entities': {'description': {'urls': []}},
 'favourites_count': 686,
 'follow_request_sent': False,
 'followers_count': 4869,
 'following': False,
 'friends_count': 4394,
 'geo_enabled': False,
 'has_extended_profile': False,
 'id': 1102182090,
 'id_str': '1102182090',
 'is_translation_enabled': False,
 'is_translator': False,
 'lang': 'ar',
 'listed_count': 8,
 'location': '',
 'name': 'شهوووده',
 'notifications': False,
 'profile_background_color': 'C0DEED',
 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_background_tile': False,
 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/1102182090/1507366778',
 'profile_image_url': 'http:/

In [35]:
data[0].head(1)

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,withheld_in_countries
0,2018-12-07 11:37:03+00:00,1071005663369814016,1071005663369814016,RT @ggeewww123: لـآ شـيء يشـبه شيء.. \nولـآ أح...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'ar', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 858138066019655680, 'id_str': '85813806...",NaN,NaN,None,NaN,{'created_at': 'Fri Dec 07 08:27:14 +0000 2018...,False,7,0,False,False,ar,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
data[1].head(1)

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status
0,2018-12-07 11:38:51+00:00,1071006116858003457,1071006116858003456,RT @shoogani2: @shahoodah33 دليل هزيمته 👌🏻💕,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'ar', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,NaN,NaN,None,"{'id': 1102182090, 'id_str': '1102182090', 'na...",None,None,None,NaN,{'created_at': 'Fri Dec 07 11:38:24 +0000 2018...,False,1,0,False,False,ar,NaN,NaN,NaN,NaN,NaN


In [37]:
print(data[0].iloc[3].text)

RT @toott4321: ،

فارقتني ؟
طيب وإذا فارقتني ؟
لا حسّ بـ موادع !
ولا اشعر بـ انكسار !!

واذا على دمعي ،
أبد ما اوجعتني 
بعض الدموع تطيح
بـ…


In [38]:
idx = np.random.choice(range(len(data[0])), size=10)
for i in idx:
    sentence = data[0].iloc[i].text
    from_lang = 'ar'
    to_lang = 'en'
    api_url = "http://mymemory.translated.net/api/get?q={}&langpair={}|{}".format(sentence, from_lang, to_lang)
    hdrs ={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive'}
    response = requests.get(api_url, headers=hdrs)
    response_json = json.loads(response.text)
    translation = response_json["responseData"]["translatedText"]
    translator_name = "MemoryNet Translation Service"
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(translation)
    print( str(vs))

{'neg': 0.18, 'neu': 0.685, 'pos': 0.134, 'compound': -0.2047}
{'neg': 0.243, 'neu': 0.629, 'pos': 0.128, 'compound': -0.4497}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.185, 'neu': 0.738, 'pos': 0.077, 'compound': -0.4404}
{'neg': 0.067, 'neu': 0.67, 'pos': 0.263, 'compound': 0.7981}
{'neg': 0.103, 'neu': 0.824, 'pos': 0.074, 'compound': -0.25}
{'neg': 0.111, 'neu': 0.746, 'pos': 0.143, 'compound': 0.2263}
{'neg': 0.133, 'neu': 0.718, 'pos': 0.148, 'compound': -0.1452}
{'neg': 0.111, 'neu': 0.746, 'pos': 0.143, 'compound': 0.2263}
{'neg': 0.111, 'neu': 0.746, 'pos': 0.143, 'compound': 0.2263}


In [41]:
idx = np.random.choice(range(len(data[1])), size=10)
for i in idx:
    sentence = data[1].iloc[i].text
    from_lang = 'ar'
    to_lang = 'en'
    api_url = "http://mymemory.translated.net/api/get?q={}&langpair={}|{}".format(sentence, from_lang, to_lang)
    hdrs ={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive'}
    response = requests.get(api_url, headers=hdrs)
    response_json = json.loads(response.text)
    translation = response_json["responseData"]["translatedText"]
    translator_name = "MemoryNet Translation Service"
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(translation)
    print(str(vs))

{'neg': 0.111, 'neu': 0.746, 'pos': 0.143, 'compound': 0.2263}
{'neg': 0.111, 'neu': 0.746, 'pos': 0.143, 'compound': 0.2263}
{'neg': 0.111, 'neu': 0.746, 'pos': 0.143, 'compound': 0.2263}
{'neg': 0.111, 'neu': 0.746, 'pos': 0.143, 'compound': 0.2263}
{'neg': 0.111, 'neu': 0.746, 'pos': 0.143, 'compound': 0.2263}
{'neg': 0.111, 'neu': 0.746, 'pos': 0.143, 'compound': 0.2263}
{'neg': 0.111, 'neu': 0.746, 'pos': 0.143, 'compound': 0.2263}
{'neg': 0.111, 'neu': 0.746, 'pos': 0.143, 'compound': 0.2263}
{'neg': 0.111, 'neu': 0.746, 'pos': 0.143, 'compound': 0.2263}
{'neg': 0.111, 'neu': 0.746, 'pos': 0.143, 'compound': 0.2263}


In [42]:
idx

array([ 6787,  6072, 15145, 18290, 12165,  1801,  4016,  2825,  5266,
        2170])

In [43]:
idx = np.random.choice(range(len(data[1])), size=10)
for i in idx:
    sentence = data[1].iloc[i].text
    from_lang = 'ar'
    to_lang = 'en'
    api_url = "http://mymemory.translated.net/api/get?q={}&langpair={}|{}".format(sentence, from_lang, to_lang)
    hdrs ={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive'}
    response = requests.get(api_url, headers=hdrs)
    response_json = json.loads(response.text)
    translation = response_json["responseData"]["translatedText"]
    translator_name = "MemoryNet Translation Service"
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(translation)
    print('{}: {:-<65} >> {:-<65} {}'.format(i, sentence, translation, vs['compound']))

16447: @idclll @l1Rst رست يغار 🤗---------------------------------------- >> MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE TRANSLATIONS FOR TODAY. NEXT AVAILABLE IN  05 HOURS 13 MINUTES 34 SECONDSVISIT HTTPS://MYMEMORY.TRANSLATED.NET/DOC/USAGELIMITS.PHP TO TRANSLATE MORE 0.2263
18454: RT @Baro_1993: أيمان لما تضحك تقفل عيونها بقوه و تلقائي يرجع راسها ل ورا و تحط يدها على فمها، أحب ضحكتها ذي جداً جداً جداً و ضحكت بهالشكل م… >> MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE TRANSLATIONS FOR TODAY. NEXT AVAILABLE IN  05 HOURS 13 MINUTES 34 SECONDSVISIT HTTPS://MYMEMORY.TRANSLATED.NET/DOC/USAGELIMITS.PHP TO TRANSLATE MORE 0.2263
11032: RT @Fatema_hfc: أصبحت مُغرم عيون وأمسيت وقلبي طريح💞💕🎧✨ https://t.co/brzFxXEeJe >> MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE TRANSLATIONS FOR TODAY. NEXT AVAILABLE IN  05 HOURS 13 MINUTES 33 SECONDSVISIT HTTPS://MYMEMORY.TRANSLATED.NET/DOC/USAGELIMITS.PHP TO TRANSLATE MORE 0.2263
6144: @_jalanazi @hfc_x6 تذكر كلامي راح يكون اخطر نوعيه موسي خله قريب من المرم

In [49]:
!pip install googletrans

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 51kB 2.9MB/s 
     |████████████████████████████████| 1.0MB 7.2MB/s 
     |████████████████████████████████| 71kB 4.6MB/s 
     |████████████████████████████████| 61kB 4.4MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp37-none-any.whl size=15737 sha256=28732b4ef1901819d41a41f901242b5710b75d075a5ae65542c5333185b9d20c
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
Successfully built googletrans


In [6]:
from googletrans import Translator

In [7]:
idx = np.random.choice(range(len(data[1])), size=10)
for i in idx:
    sentence = data[1].iloc[i].text
    from_lang = 'ar'
    to_lang = 'en'
    translator = Translator()
    translation = translator.translate(sentence, dest=to_lang)
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(translation)
    print('{}: {:-<35} >> {:-<35} {}'.format(i, sentence, translation, vs['compound']))

AttributeError: ignored

In [55]:
type(data[1].iloc[i].text)

str

In [59]:
translator = Translator()
translation = translator.translate(data[0].iloc[0].text, dest=to_lang)

AttributeError: ignored

In [62]:
data[0].iloc[0].text

'RT @ggeewww123: لـآ شـيء يشـبه شيء.. \nولـآ أحد يعوّض غيآب أحد 💔\nأشياؤنـآ ﺂلمُختلفـة إن ضـآعت..\n لـيسَ لهـآ بـديـل💭🔕 \n\n🔹 🔹'

In [64]:
translator = Translator()
translator.translate('لـيسَ لهـآ بـديـل')

AttributeError: ignored

In [67]:
translator = Translator()
translator.translate('안녕하세요.')

AttributeError: ignored

In [68]:
!pip install git+https://github.com/BoseCorp/py-googletrans.git --upgrade

  Cloning https://github.com/BoseCorp/py-googletrans.git to /tmp/pip-req-build-xkp_iidx
  Running command git clone -q https://github.com/BoseCorp/py-googletrans.git /tmp/pip-req-build-xkp_iidx
  Created wheel for googletrans: filename=googletrans-2.3.0-cp37-none-any.whl size=16449 sha256=45a81ce1d51aef3a336bfa219d79726a0712db2ddd3e35f7e45d6d0a157f5066
  Stored in directory: /tmp/pip-ephem-wheel-cache-or1pail2/wheels/6a/fc/9e/2d31d95d9e97da5166afd8225a6f3b6850dc2c6e84accefbfc
Successfully built googletrans
  Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0


In [8]:
!pip uninstall googletrans
!git clone https://github.com/alainrouillon/py-googletrans.git
!cd ./py-googletrans
!git checkout origin/feature/enhance-use-of-direct-api
!python setup.py install

Uninstalling googletrans-2.3.0:
  Would remove:
    /usr/local/bin/translate
    /usr/local/lib/python3.7/dist-packages/googletrans-2.3.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/googletrans/*
Proceed (y/n)? y
  Successfully uninstalled googletrans-2.3.0
Cloning into 'py-googletrans'...
remote: Enumerating objects: 633, done.
remote: Total 633 (delta 0), reused 0 (delta 0), pack-reused 633
Receiving objects: 100% (633/633), 582.18 KiB | 4.28 MiB/s, done.
Resolving deltas: 100% (375/375), done.
fatal: not a git repository (or any of the parent directories): .git
python3: can't open file 'setup.py': [Errno 2] No such file or directory


In [9]:
!git checkout origin/feature/enhance-use-of-direct-api

fatal: not a git repository (or any of the parent directories): .git


In [10]:
!python setup.py install

python3: can't open file 'setup.py': [Errno 2] No such file or directory


In [11]:
!ls

drive  py-googletrans  sample_data


In [13]:
!python py-googletrans/setup.py install

running install
running bdist_egg
running egg_info
creating googletrans.egg-info
writing googletrans.egg-info/PKG-INFO
writing dependency_links to googletrans.egg-info/dependency_links.txt
writing requirements to googletrans.egg-info/requires.txt
writing top-level names to googletrans.egg-info/top_level.txt
writing manifest file 'googletrans.egg-info/SOURCES.txt'
reading manifest file 'googletrans.egg-info/SOURCES.txt'
writing manifest file 'googletrans.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
installing scripts to build/bdist.linux-x86_64/egg/EGG-INFO/scripts
running install_scripts
running build_scripts
creating build/scripts-3.7
error: file '/content/translate' does not exist


In [14]:
!pip install translate

     |████████████████████████████████| 194kB 6.2MB/s 
     |████████████████████████████████| 92kB 6.9MB/s 
     |████████████████████████████████| 7.2MB 46.1MB/s 
     |████████████████████████████████| 645kB 20.9MB/s 
     |████████████████████████████████| 102kB 6.7MB/s 
     |████████████████████████████████| 337kB 32.7MB/s 
ERROR: pytest 3.6.4 has requirement pluggy<0.8,>=0.5, but you'll have pluggy 0.13.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1


In [19]:
from translate import Translator
translator= Translator(to_lang='en')
translator.translate('Ich kann kein Deutsch.')

'Ich kann kein Deutsch.'

In [20]:
!pip install translators

     |████████████████████████████████| 5.5MB 6.6MB/s 
     |████████████████████████████████| 61kB 4.3MB/s 
  Created wheel for PyExecJS: filename=PyExecJS-1.5.1-cp37-none-any.whl size=14588 sha256=edb7a238c8ad3a67440485e8adc585ef12a6e636f41302419e90dfce0f7c81f4
  Stored in directory: /root/.cache/pip/wheels/43/25/28/52dd7a6c691b1551e3d4482f3f16ef630cc9f59cae99cd33ba
Successfully built PyExecJS
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [8]:
import translators as ts

# input languages
print(ts.bing('Ich kann kein Deutsch.')) # default: from_language='auto', to_language='en'

I can't German.


In [ ]:
idx = np.random.choice(range(len(data[1])), size=10)

In [10]:
values = []
for i in idx:
    sentence = data[1].iloc[i].text
    from_lang = 'ar'
    to_lang = 'en'
    translation = ts.google(sentence)
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(translation)
    values.append(vs)
    print('{}: {:-<35} >> {:-<35} {}'.format(i, sentence, translation, vs['compound']))

3298: RT @MULTI_MACA01: #توقع_النتيجه_الهلال_والنصر

🌷البديل الأقوى للفياجرا 
💥مالتى ماكا💥
❤لحياة زوجية سعيدة❤

💥 لزيادة الإنتصاب 

💥 لزيادة الرغ… >> Rt @ multi_maca01: # Expect_Stegraph_الهال_النصر

🌷 The strongest alternative to Vigra
MAL MACA
Life is a happy married

💥 To increase settlement

💥 To increase the ... 0.7351
5737: لا أحد يشاركك الحزن ولا أحد يشاركك دمعتك لهذا!اصنع لنفسك مجداً وكُن سعيداً❤❤✔ >> No one shares you sadness and no one share your tears for this! Make yourself glory and be happy .❤✔ 0.4051
17494: RT @AlragiM66: ف وقتها الاهلي اخذ الدوري
والحين الاتحاد بيهبط :)
#هلا_بالخميس
 https://t.co/mY44xG7rNR >> RT @ alragim66: The father of Ahli takes the league
And the union is lining :)
# Hala_akhams
 https://t.co/my44xg7rnr 0.4588
1458: RT @Pythagoras_y: @qlmMCB1 @00Benyameen @V2bivR1AleC3ZwA @TawfikMoshreh @zVJHYa9Iz8UaZT6 @heyam2255 مرة ثانيع خلي @zVJHYa9Iz8UaZT6  مفحطة ب… >> Rt @pythagoras_y: @ qlmmcb1 @ 00benyameen @ v2bivr1alec3zwa @tawfikmoshreh @ zvjhya9iz8uazt

In [11]:
values

[{'compound': 0.7351, 'neg': 0.134, 'neu': 0.565, 'pos': 0.301},
 {'compound': 0.4051, 'neg': 0.326, 'neu': 0.393, 'pos': 0.28},
 {'compound': 0.4588, 'neg': 0.0, 'neu': 0.857, 'pos': 0.143},
 {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0},
 {'compound': 0.5994, 'neg': 0.0, 'neu': 0.855, 'pos': 0.145},
 {'compound': 0.3987, 'neg': 0.164, 'neu': 0.583, 'pos': 0.253},
 {'compound': 0.5139, 'neg': 0.062, 'neu': 0.794, 'pos': 0.144},
 {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0},
 {'compound': 0.5574, 'neg': 0.0, 'neu': 0.66, 'pos': 0.34},
 {'compound': 0.0857, 'neg': 0.082, 'neu': 0.797, 'pos': 0.121}]

In [12]:
for i in idx:
    sentence = data[0].iloc[i].text
    from_lang = 'ar'
    to_lang = 'en'
    translation = ts.google(sentence)
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(translation)
    values.append(vs)
    print('{}: {:-<35} >> {:-<35} {}'.format(i, sentence, translation, vs['compound']))

3298: RT @Sdad__2: ⁧#تسديد_القروض_وتعثرات_سمه⁩
.
🎯💸سداد متعثرات
.
🎯💸سداد القرض القديم
.
🎯💸استخراج جديد

🎯💸من 18 ⁧#راتب⁩ الى 22 راتب
.
🎯💸⁧#مدنيين⁩… >> Rt @ sdad__2: # Payment_القرار_المواضات_موعه
.
Pastest extremes
.
The old loan
.
New extraction

🎯💸🎯💸 18 # Salary to 22 Salary
.
🎯💸 # civilians ... 0.0
5737: RT @Science998: هل تؤيد سالفة حجز المواد ؟و لماذا؟
من وجهة نظري الشخصية لا لان صار الاختياريات ما انقدر نحصلهن لان ما شاء الله اول دفعة تسج… >> RT @ science998: Does he support the reservation of materials? And why?
From my personal point of view, not because the options have become what we have lost because what God wills the first boost ... 0.6767
17494: @3FeGrFC1AHj0Kz0 ☹شكلي ما يبين لون الاحمر لوني لليل  داكن غامق☹🙁 >> @ 3fegrfc1Ahj0kz0 ☹Chly what shows the color of the red color for a dark dark -0.34
1458: RT @Eslam__3bdallah: مجتش عليا 🤷‍♂️👀 https://t.co/geTdWV51Ua >> RT @ Eslam__3bdallah: Alia Alia 🤷♂️👀 https://t.co/getdwv51ua 0.0
3842: RT @moj2200: 💡كيف تنقذ نفسك او موكلك من 

In [13]:
values[10:]

[{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0},
 {'compound': 0.6767, 'neg': 0.055, 'neu': 0.759, 'pos': 0.186},
 {'compound': -0.34, 'neg': 0.138, 'neu': 0.862, 'pos': 0.0},
 {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0},
 {'compound': 0.1779, 'neg': 0.061, 'neu': 0.86, 'pos': 0.079},
 {'compound': 0.6705, 'neg': 0.082, 'neu': 0.74, 'pos': 0.178},
 {'compound': -0.802, 'neg': 0.505, 'neu': 0.38, 'pos': 0.114},
 {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0},
 {'compound': -0.25, 'neg': 0.403, 'neu': 0.325, 'pos': 0.273},
 {'compound': 0.9571, 'neg': 0.217, 'neu': 0.401, 'pos': 0.382}]